Author: Samhaa El-Beltagy

In [ ]:
!pip install langchain==0.0.275
!pip install openai
!pip install tiktoken

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import os
import os
import openai
import sys
sys.path.append('../..')

In [ ]:

def calculate_average_question_length_in_words(df, qfieldName):
    # Read the CSV file into a DataFrame

    # Calculate the total number of words in the questions
    total_words = df[qfieldName].str.split().apply(len).sum()

    # Count the number of questions
    question_count = df[qfieldName].count()

    # Calculate the average length in words
    if question_count == 0:
        return 0
    average_length_in_words = total_words / question_count

    return average_length_in_words

# Example usage

file_path = 'drive/My Drive/Colab Notebooks/Rag_Test/ACLing/allData.csv'
df = pd.read_csv(file_path)
average_length_in_words = calculate_average_question_length_in_words(df, 'question')
print(f"The average length of ARCD questions in words is: {average_length_in_words}")
file_path = 'drive/My Drive/Colab Notebooks/Rag_Test/ACLing/Retrieval_Data.csv'
df2 = pd.read_csv(file_path)
average_length_in_words = calculate_average_question_length_in_words(df2, 'Questions')
print(f"The average length of custom DS questions in words is: {average_length_in_words}")

The average length of ARCD questions in words is: 6.230107526881721
The average length of custom DS questions in words is: 9.537974683544304


In [ ]:
q = df['question']


In [ ]:
x = q.iloc[704:714]
x

704            في أي مدينة ولد جمال أحمد حمزة خاشقجي? ال
705       - في أي صحيفة قام بكتابة عمود منذ عام 2017؟ ال
706        كيف وصفها في الصحف ووسائل الإعلام الدولية? ال
707                        في أي منصب شغل في الجريدة? ال
708                         متى غادر خاشقجي السعودية? ال
709                  لمن ينتقده في مقالاته الإخبارية? ال
710       لمن انتقد خاشقجي بشدة في مقالاته الإخبارية? ال
711             أي دولة هي أكبر دولة في الشرق الأوسط? ال
712         أين تقع المملكة العربية السعودية في آسيا? ال
713    ما هي مساحة الجزء الأكبر من شبه الجزيرة العربي...
Name: question, dtype: object

In [ ]:
llm_name = "gpt-3.5-turbo"

In [ ]:
api_key_path = '/content/drive/My Drive/Colab Notebooks/Rag_Test/openApiKey.txt'

with open(api_key_path, 'r') as file:
    openai_api_key = file.read().strip()

# Set the environment variable
os.environ['OPENAI_API_KEY'] = openai_api_key

# Verify that the environment variable is set correctly

openai.api_key  = os.environ['OPENAI_API_KEY']

In [ ]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
os.environ["LANGCHAIN_API_KEY"] = "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"


In [ ]:
from langchain.chat_models import ChatOpenAI
llm = OpenAI(model_name=llm_name, temperature=0)

/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:200: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:787: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [ ]:
from openai import OpenAI
client = OpenAI()

def format_prompt(q1, q2):
    return f"Question 1: {q1}, Question 2: {q2}. Do not modify question 2 unless it has an ambigious entity that needs to be determined. Any name, even if incomplete is not considered ambigious"

def getModifiedQuestion(q1, q2):
    prompt = format_prompt(q1, q2)

    prom = [
        {"role": "system", "content": "You are someone who understands Arabic well. Given two questions, rephrase the second question to resolve any ambiguity using the first question. If there is no ambguity, return the text of question 2 as is and do not add any extra text."},
        {"role": "user", "content": prompt}
      ]

    completion = client.chat.completions.create(
      model=llm_name,
      messages=prom,
      temperature=0

    )
    return completion.choices[0].message.content

In [ ]:
def format_prompt_amb(q):
    return f"Question: {q}"

def getAmb(q):
    prompt = format_prompt_amb(q)
    print(prompt)
    prom = [
        {"role": "system", "content": "You are someone who understands Arabic well. Does the following Arabic question contain an anaphora reference. Only answer with yes or no. ."},
        {"role": "user", "content": prompt}
      ]

    completion = client.chat.completions.create(
      model=llm_name,
      messages=prom,
      temperature=0

    )
    return completion.choices[0].message.content

In [ ]:
getAmb(q.iloc[0])

Question: من هو حمزة بن عبد المطلب؟


'Yes.'

In [ ]:
mq = q.iloc[0]
for i in range(1, len(q)):
    mq = getModifiedQuestion(mq, q.iloc[i])
    print(q.iloc[i], mq)
    modified_questions.append(mq)

بما وصفه رسول الله؟ بما وصف حمزة بن عبد المطلب رسول الله؟
بما وصف رسول الله على ؟ بما وصف رسول الله حمزة بن عبد المطلب؟
متى اسلم حمزة؟ متى اسلم حمزة بن عبد المطلب؟
و ماذا فعل فى غزوة بدر؟ Question 2: و ماذا فعل حمزة بن عبد المطلب فى غزوة بدر؟.
من قتل حمزة؟ Question 2: من قتل حمزة بن عبد المطلب؟.
من هو عبد المطلب بن هاشم؟ Question 2: من هو عبد المطلب بن هاشم؟.
من اخو حمزة فى الرضاعة؟ Question 2: من اخو عبد المطلب بن هاشم فى الرضاعة؟.
ماذا جمع بين  حمزة و محمد بن عبد الله؟ Question 2: ماذا جمع بين حمزة و محمد بن عبد الله؟.
ما هو القمر؟ Question 2: ما هو القمر؟.
كم يبلغ قطره؟ Question 2: ما هو قطر القمر؟.
م تبلغ كتلته؟ Question 2: ما هو قطر القمر؟.
ما هى سمات القمر؟ Question 2: ما هي سمات القمر؟.
كم المسافة المدارية للقمر؟ Question 2: ما هي المسافة المدارية للقمر؟.
ما اللذى يتسبب فى ظهور القمر بنفس حجمه تقريبا؟ Question 2: ما اللذى يتسبب فى ظهور القمر بنفس حجمه تقريبا؟.
لمن اول سفينة فضائية وصلت للقمر؟ لمن اول سفينة فضائية وصلت للقمر؟
متى وصلت اول بعثة بشرية للقمر؟ Question 1: لمن اول سفي

In [ ]:
modified_questions[1:14]

['بما وصف حمزة بن عبد المطلب رسول الله؟',
 'بما وصف رسول الله حمزة بن عبد المطلب؟',
 'متى اسلم حمزة بن عبد المطلب؟',
 'Question 2: و ماذا فعل حمزة بن عبد المطلب فى غزوة بدر؟.',
 'Question 2: من قتل حمزة بن عبد المطلب؟.',
 'Question 2: من هو عبد المطلب بن هاشم؟.',
 'Question 2: من اخو عبد المطلب بن هاشم فى الرضاعة؟.',
 'Question 2: ماذا جمع بين حمزة و محمد بن عبد الله؟.',
 'Question 2: ما هو القمر؟.',
 'Question 2: ما هو قطر القمر؟.',
 'Question 2: ما هو قطر القمر؟.',
 'Question 2: ما هي سمات القمر؟.',
 'Question 2: ما هي المسافة المدارية للقمر؟.']

In [ ]:
q.iloc[1:14]

1                        بما وصفه رسول الله؟
2                    بما وصف رسول الله على ؟
3                             متى اسلم حمزة؟
4                    و ماذا فعل فى غزوة بدر؟
5                               من قتل حمزة؟
6                  من هو عبد المطلب بن هاشم؟
7                    من اخو حمزة فى الرضاعة؟
8     ماذا جمع بين  حمزة و محمد بن عبد الله؟
9                               ما هو القمر؟
10                             كم يبلغ قطره؟
11                             م تبلغ كتلته؟
12                         ما هى سمات القمر؟
13                كم المسافة المدارية للقمر؟
Name: question, dtype: object

In [ ]:
df['modified_question'] = modified_questions

In [ ]:
df

,id,title,context,question,answers,modified_question
0,6.217230e+11,حمزة بن عبد المطلب,حمزة بن عبد المطلب الهاشمي القرشي صحابي من صحا...,من هو حمزة بن عبد المطلب؟,{'text': array(['صحابي من صحابة رسول الإسلام م...,من هو حمزة بن عبد المطلب؟
1,1.891050e+11,حمزة بن عبد المطلب,حمزة بن عبد المطلب الهاشمي القرشي صحابي من صحا...,بما وصفه رسول الله؟,"{'text': array(['وَخَيْرُ أَعْمَامِي'], dtype=...",بما وصف حمزة بن عبد المطلب رسول الله؟
2,6.626170e+11,حمزة بن عبد المطلب,حمزة بن عبد المطلب الهاشمي القرشي صحابي من صحا...,بما وصف رسول الله على ؟,"{'text': array(['«خَيْرُ إِخْوَتِي عَلِيٌّ،'],...",بما وصف رسول الله حمزة بن عبد المطلب؟
3,5.014659e+10,حمزة بن عبد المطلب,أسلم حمزة في السنة الثانية من بعثة النبي محمد،...,متى اسلم حمزة؟,{'text': array(['في السنة الثانية من بعثة النب...,متى اسلم حمزة بن عبد المطلب؟
4,2.595640e+11,حمزة بن عبد المطلب,أسلم حمزة في السنة الثانية من بعثة النبي محمد،...,و ماذا فعل فى غزوة بدر؟,{'text': array(['وقَتَلَ فيها شيبة بن ربيعة مب...,Question 2: و ماذا فعل حمزة بن عبد المطلب فى غ...
...,...,...,...,...,...,...
1390,1.880900e+11,بظر,البظر هو المنطقة المثيرة للشهوة الجنسية الأكثر...,ما هو حجم رأس البظر البشرى؟,{'text': array(['هو تقريبًا كحجم وشكل حبة البا...,Question 2: ما هو حجم رأس البظر البشرى؟
1391,6.714452e+10,بظر,البظر هو المنطقة المثيرة للشهوة الجنسية الأكثر...,كم عدد النهايات العصبية فى رأس البظر البشرى؟,"{'text': array(['أكثر من 8000'], dtype=object)...",Question 2: كم عدد النهايات العصبية في رأس الب...
1392,7.030490e+11,بظر,"يقول قاموس إكسفورد للغة الإنجليزيّة أن لكلمة ""...","ما هو اصل كلمة""Clitoris""طبقا لقاموس اكسفورد لل...","{'text': array(['اللغة الإغريقية القديمة'], dt...","ما هو أصل كلمة ""Clitoris"" طبقا لقاموس أكسفورد ..."
1393,8.729660e+11,بظر,"يقول قاموس إكسفورد للغة الإنجليزيّة أن لكلمة ""...","اين استخدم الشكل المختصر ::Clit""؟","{'text': array(['في الولايات المتحدة'], dtype=...","Where is the shortened form ""Clit"" used?"


In [ ]:
#save df to .csv file
df.to_csv('drive/My Drive/Colab Notebooks/Rag_Test/ACLing/allData_mod.csv', encoding='UTF8')

In [ ]:
#download file to local computer
from google.colab import files
files.download('drive/My Drive/Colab Notebooks/Rag_Test/ACLing/allData_mod.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def is_arabic(text):
    """ Check if the majority of the text is Arabic """
    arabic_count = sum('ء' <= c <= 'ي' for c in text)
    return arabic_count > len(text) / 2

In [ ]:
import re

def extract_question_2(text):
    # Define a regular expression pattern to find 'Question 2' and its following text
    pattern = r'Question 2: (.*)'

    # Search for the pattern in the text
    match = re.search(pattern, text)

    # If a match is found, return the matched group which corresponds to the text of Question 2
    if match:
        return match.group(1)
    else:
        return text


In [ ]:
def format_prompt(q1, q2):
    return f"""
    Question 1: {q1}
    Question 2: {q2}

    You are an expert who understands Arabic fluently. Given these two questions, your task is to rephrase the second question only if it contains ambiguities that might confuse someone without context from the first question. An ambiguity might be a vague reference or unclear term that cannot be understood without additional context. Do not modify question 2 if it is clear and understandable on its own. Always maintain the response in Arabic.

    Example 1:
    Question 1: من هو حمزة بن عبد المطلب؟
    Question 2: بما وصفه رسول الله؟
    Correct modification: بما وصف رسول الله حمزة بن عبد المطلب ؟

    Example 2:
    Question 1: كم يبلغ ارتفاع مكة عن سطح البحر؟
    Question 2: اين تقع مكة؟
    Correct modification: اين تقع مكة؟

    Remember, modifications are only needed if they clarify ambiguities directly related to the context provided by question 1. Any name or specific noun is not considered ambiguous.
    """

def get_modified_question(q1, q2):
    prompt = format_prompt(q1, q2)
    messages = [
        {"role": "system", "content": "Given two questions, ensure that the second question is unambiguous and clear in Arabic based on the context of the first question. Modify only if necessary."},
        {"role": "user", "content": prompt}
    ]

    # Adjust the completion settings as per your API usage and specifications
    completion = client.chat.completions.create(
        model=llm_name,
        messages=messages,
        temperature=0.0  # Lower temperature for less variability
    )

    modified_q2 = completion.choices[0].message.content
    result = {
        "original_question": q2,
        "modified_question": extract_question_2(modified_q2),
        "changed": modified_q2 != q2 and not q2.startswith(modified_q2)
    }

    return result


In [ ]:
mod2 = []
mod2.append(q.iloc[0])


In [ ]:
mq = q.iloc[701]
#for i in range(1, len(q)):
for i in range(702, len(df)):
    rec = get_modified_question(mq, q.iloc[i])
    mq = rec['modified_question']
    print(i, q.iloc[i], mq)
    if(is_arabic(mq)):
        mod2.append(mq)
    else:
        mod2.append(q.iloc[i])

702 من هو جمال أحمد حمزة خاشقجي? من هو جمال أحمد حمزة خاشقجي؟
703 متى ولد جمال أحمد حمزة خاشقجي وتوفي? ال متى ولد جمال أحمد حمزة خاشقجي ومتى توفي؟
704 في أي مدينة ولد جمال أحمد حمزة خاشقجي? ال في أي مدينة ولد جمال أحمد حمزة خاشقجي؟
705 - في أي صحيفة قام بكتابة عمود منذ عام 2017؟ ال - في أي صحيفة قام بكتابة عمود منذ عام 2017؟
706 كيف وصفها في الصحف ووسائل الإعلام الدولية? ال كيف وصفها في الصحف ووسائل الإعلام الدولية؟
707 في أي منصب شغل في الجريدة? ال في أي منصب شغلها في الجريدة؟
708 متى غادر خاشقجي السعودية? ال متى غادر جمال خاشقجي السعودية؟
709 لمن ينتقده في مقالاته الإخبارية? ال من ينتقده في مقالاته الإخبارية؟
710 لمن انتقد خاشقجي بشدة في مقالاته الإخبارية? ال لمن انتقد جمال خاشقجي بشدة في مقالاته الإخبارية؟
711 أي دولة هي أكبر دولة في الشرق الأوسط? ال ما هي أكبر دولة في الشرق الأوسط؟
712 أين تقع المملكة العربية السعودية في آسيا? ال أين تقع المملكة العربية السعودية في آسيا؟
713 ما هي مساحة الجزء الأكبر من شبه الجزيرة العربية? ال ما هي مساحة الجزء الأكبر من شبه الجزيرة العربية؟
714 ما 

In [ ]:
x = get_modified_question(mq, q.iloc[1])


In [ ]:
x

{'original_question': 'بما وصفه رسول الله؟',
 'modified_question': 'Question 1: من هو حمزة بن عبد المطلب؟\nQuestion 2: بما وصف رسول الله حمزة بن عبد المطلب؟',
 'changed': True}

In [ ]:
y = extract_question_2(x['modified_question'])

In [ ]:
z = get_modified_question(z, q.iloc[3])
z


{'original_question': 'متى اسلم حمزة؟',
 'modified_question': 'متى اسلم حمزة بن عبد المطلب؟',
 'changed': True}

In [ ]:
len(mod2)

702

In [ ]:
q.iloc[702]

'من هو جمال أحمد حمزة خاشقجي?'

In [ ]:
mod2[702:]=[]


In [ ]:
df['modified_question'] = mod2